# ASSIGNMENT 5
Leen Said 2220356194 

In [1]:
import numpy as np
import pandas as pd

In [2]:
p=pd.read_csv("C:/Users/leens/Downloads/16P.csv",header=0, encoding='cp1252')
df=pd.DataFrame(p)
df['Personality'].replace("ESTJ",0 , inplace=True)
df['Personality'].replace("ENTJ",1 , inplace=True)
df['Personality'].replace("ESFJ",2 , inplace=True)
df['Personality'].replace("ENFJ",3 , inplace=True)
df['Personality'].replace("ISTJ",4 , inplace=True)
df['Personality'].replace("ISFJ",5 , inplace=True)
df['Personality'].replace("INTJ",6 , inplace=True)
df['Personality'].replace("INFJ",7 , inplace=True)
df['Personality'].replace("ESTP",8 , inplace=True)
df['Personality'].replace("ESFP",9 , inplace=True)
df['Personality'].replace("ENTP",10 , inplace=True)
df['Personality'].replace("ENFP",11 , inplace=True)
df['Personality'].replace("ISTP",12 , inplace=True)
df['Personality'].replace("ISFP",13 , inplace=True)
df['Personality'].replace("INTP",14 , inplace=True)
df['Personality'].replace("INFP",15 , inplace=True)
df=df.drop(['Response Id'], axis=1)

clist = list(df.columns)
clist_pred, clist_target = clist[:-1], clist[-1]
independent_df, dependent_df = df[clist_pred], df[clist_target] 
independent_array, dependent_array, main = independent_df.to_numpy() , dependent_df.to_numpy(), df.to_numpy()

In [3]:
def normalize_dataframe(X_train, X_test):
    # Normalize each column in the DataFrame
    X_train = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
    X_test = (X_test - X_test.min(axis=0)) / (X_test.max(axis=0) - X_test.min(axis=0))
    return X_train, X_test

In [4]:
def accuracy_score(y_true, y_pred):
    # Initialize a variable to store the number of correct predictions
    correct = 0
    
    # Iterate over each element in y_true
    for i in range(len(y_true)):
        # Check if the prediction for the current element is correct
        if y_true[i] == y_pred[i]:
            # If the prediction is correct, increment the correct variable by 1
            correct += 1
    # Return the proportion of correct predictions
    return correct / len(y_true)

def precision_score(y_true, y_pred):
    # Initialize a dictionary to store the number of true positive for each class label
    true_positive = {k: 0 for k in set(y_true)}
    # Initialize a dictionary to store the number of false positive for each class label
    false_positive = {k: 0 for k in set(y_true)}
    # Iterate over the true and predicted class labels
    for i in range(len(y_true)):
        # If the true and predicted class labels match
        if y_true[i] == y_pred[i]:
            # Increment the count of true positive for that class label
            if y_true[i] not in true_positive:
                true_positive[y_true[i]] = 1
            else:
                true_positive[y_true[i]] += 1
        # If the true and predicted class labels do not match
        else:
            # Increment the count of false positive for that class label
            if y_pred[i] not in false_positive:
                false_positive[y_pred[i]] = 1
            else:
                false_positive[y_pred[i]] += 1
    # Initialize an empty list to store precision for each class label
    precision = []
    # Calculate precision for each class label
    for label in set(y_true):
        if true_positive[label] + false_positive[label] != 0:  # check for division by zero
            precision.append(true_positive[label] / (true_positive[label] + false_positive[label]))
    # Return the mean precision
    return np.mean(precision)


def recall_score(y_true, y_pred):
    # Initialize a dictionary to store the number of true positive for each class label
    true_positive = {k: 0 for k in set(y_true)}
    # Initialize a dictionary to store the number of false negative for each class label
    false_negative = {k: 0 for k in set(y_true)}
    # Iterate over the true and predicted class labels
    for i in range(len(y_true)):
        # If the true and predicted class labels match
        if y_true[i] == y_pred[i]:
            # Increment the count of true positive for that class label
            if y_true[i] not in true_positive:
                true_positive[y_true[i]] = 1
            else:
                true_positive[y_true[i]] += 1
        # If the true and predicted class labels do not match
        else:
            # Increment the count of false negative for that class label
            if y_true[i] not in false_negative:
                false_negative[y_true[i]] = 1
            else:
                false_negative[y_true[i]] += 1
    # Initialize an empty list to store recall for each class label
    recall = []
    # Calculate recall for each class label
    for label in set(y_true):
        recall.append(true_positive[label] / (true_positive[label] + false_negative[label]))
    # Return the mean recall
    return np.mean(recall)

In [5]:
class KNeighborsClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    def fit_with_norm(self, X_train, y_train):
        self.X_train, self.X_test = normalize_dataframe(X_train, X_test)
        self.y_train = y_train

    def predict(self, X_test):
        # Initialize an empty list to store predictions
        predictions = []
        
        # Iterate over each feature vector in X_test
        for i in range(len(X_test)):
            # Initialize an empty list to store distances
            distances = []
            
            # Calculate the Euclidean distance between the current feature vector in X_test and all feature vectors in X_train
            for j in range(len(self.X_train)):
                dist = sum([(a-b)**2 for a,b in zip(X_test[i],self.X_train[j])])
                distances.append([dist, j])
            
            # Sort the distances in ascending order and select the first 'n_neighbors' closest distances
            distances = sorted(distances, key=lambda x: x[0])[:self.n_neighbors]
            
            # Initialize an empty list to store the class labels of the closest neighbors
            neighbors = []
            
            # Add the class labels of the closest neighbors to the "neighbors" list
            for k in range(self.n_neighbors):
                index = distances[k][1]
                neighbors.append(self.y_train[index])
            
            # Find the class label that appears most frequently in the "neighbors" list
            # and append it to the "predictions" list as the prediction for the current feature vector
            predictions.append(max(set(neighbors), key=neighbors.count))
            
        # Return the final list of predictions
        return predictions


# Define a list of possible values of k to try
k_list = [1, 3, 5, 7, 9]

# Use cross-validation to evaluate the performance of k-NN with different values of k
cv_scores = []
cv_scores_with_norm = []
# Define the number of folds for cross-validation
n_folds = 5

for k_fold in k_list:
    # Split the data into n_folds subsets
    subsets = np.array_split(np.arange(len(independent_array)), n_folds)
    k_scores = []
    k_scores_with_norm = []
    for i in range(n_folds):
        # Use one subset for testing and the others for training
        test_indices = subsets[i]
        train_indices = np.concatenate(subsets[:i] + subsets[i+1:])
        X_train, y_train = independent_array[train_indices], dependent_array[train_indices]
        X_test, y_test = independent_array[test_indices], dependent_array[test_indices]


    # Create a KNeighborsClassifier object with k_fold
    knn = KNeighborsClassifier(n_neighbors=k_fold)

    # Fit the classifier to the training data
    knn.fit(X_train, y_train)

    # Make predictions on the test set
    predictions_knn = knn.predict(X_test)
    print("Before normalization : ")
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions_knn)
    print("Accuracy for k = {}:".format(k_fold), accuracy)

    # Calculate precision
    precision = precision_score(y_test, predictions_knn)
    print("Precision for k = {}:".format(k_fold), precision)

    # Calculate recall
    recall = recall_score(y_test, predictions_knn)
    print("Recall for k = {}:".format(k_fold), recall,'\n')
    k_scores.append(accuracy)
    cv_scores.append(np.mean(k_scores))

    # with NORMALIZATION
    print("After normalization : ")
    knn.fit_with_norm(X_train, y_train)
    X_test, _ = normalize_dataframe(X_test, X_train)
    # Make predictions on the test set
    predictions_knn_with_norm = knn.predict(X_test)

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions_knn_with_norm)
    print("Accuracy for k = {}:".format(k_fold), accuracy)

    # Calculate precision
    precision = precision_score(y_test, predictions_knn_with_norm)
    print("Precision for k = {}:".format(k_fold), precision)

    # Calculate recall
    recall = recall_score(y_test, predictions_knn_with_norm)
    print("Recall for k = {}:".format(k_fold), recall,'\n')
    # evaluate predictions
    score_with_normalization = accuracy_score(y_test, predictions_knn_with_norm)
    k_scores_with_norm.append(score_with_normalization)
    cv_scores_with_norm.append(sum(k_scores_with_norm)/len(k_scores_with_norm))

Before normalization : 
Accuracy for k = 1: 0.9266666666666666
Precision for k = 1: 0.9231903492391109
Recall for k = 1: 0.9289286281851103 

After normalization : 
Accuracy for k = 1: 0.8966666666666666
Precision for k = 1: 0.8997205316613212
Recall for k = 1: 0.8971835849801213 

Before normalization : 
Accuracy for k = 3: 0.9533333333333334
Precision for k = 3: 0.9530636688503068
Recall for k = 3: 0.9535878167133005 

After normalization : 
Accuracy for k = 3: 0.92
Precision for k = 3: 0.9204365778345285
Recall for k = 3: 0.9189922907115244 

Before normalization : 
Accuracy for k = 5: 0.97
Precision for k = 5: 0.9684366246498599
Recall for k = 5: 0.9709004519946858 

After normalization : 
Accuracy for k = 5: 0.9533333333333334
Precision for k = 5: 0.9506444089324524
Recall for k = 5: 0.9540849758042096 

Before normalization : 
Accuracy for k = 7: 0.98
Precision for k = 7: 0.9808358990147783
Recall for k = 7: 0.9798942907582613 

After normalization : 
Accuracy for k = 7: 0.95
Pre

In [6]:
class WeightedKNeighborsClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    def fit_with_norm(self, X_train, y_train):
        self.X_train, self.X_test = normalize_dataframe(X_train, X_test)
        self.y_train = y_train

    def predict(self, X_test):
        # Initialize an empty list to store predictions
        predictions = []

        # Iterate over each feature vector in X_test
        for i in range(len(X_test)):
            # Initialize an empty list to store distances
            distances = []

            # Calculate the Euclidean distance between the current feature vector in X_test and all feature vectors in X_train
            for j in range(len(self.X_train)):
                dist = sum([(a-b)**2 for a,b in zip(X_test[i],self.X_train[j])])
                distances.append([dist, j])

            # Sort the distances in ascending order and select the first 'n_neighbors' closest distances
            distances = sorted(distances, key=lambda x: x[0])[:self.n_neighbors]

            # Initialize a dictionary to store the class labels of the closest neighbors with their corresponding weight
            neighbors = {}

            # Calculate the weight for each class label of the closest neighbors
            for k in range(self.n_neighbors):
                index = distances[k][1]
                weight = 1 / (distances[k][0] + 0.00001) # weight = 1 / distance + epsilon to prevent division by zero
                if self.y_train[index] in neighbors:
                    neighbors[self.y_train[index]] += weight
                else:
                    neighbors[self.y_train[index]] = weight

            # Find the class label with the highest weighted vote and append it to the "predictions" list as the prediction for the current feature vector
            predictions.append(max(neighbors, key=neighbors.get))

        # Return the final list of predictions
        return predictions


for k_fold in k_list:
    # Split the data into n_folds subsets
    subsets = np.array_split(np.arange(len(independent_array)), n_folds)
    k_scores = []
    for i in range(n_folds):
        # Use one subset for testing and the others for training
        test_indices = subsets[i]
        train_indices = np.concatenate(subsets[:i] + subsets[i+1:])
        X_train, y_train = independent_array[train_indices], dependent_array[train_indices]
        X_test, y_test = independent_array[test_indices], dependent_array[test_indices]

    # Create a KNeighborsClassifier object with k_fold
    knn = WeightedKNeighborsClassifier(n_neighbors=k_fold)

    # Fit the classifier to the training data
    knn.fit(X_train, y_train)

    # Make predictions on the test set
    predictions_W_knn = knn.predict(X_test)
    print("Before Normalization : ")
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions_W_knn)
    print("Accuracy for k = {}:".format(k_fold), accuracy)

    # Calculate precision
    precision = precision_score(y_test, predictions_W_knn)
    print("Precision for k = {}:".format(k_fold), precision)

    # Calculate recall
    recall = recall_score(y_test, predictions_W_knn)
    print("Recall for k = {}:".format(k_fold), recall,"\n")
    k_scores.append(accuracy)
    cv_scores.append(np.mean(k_scores))
    
    # with NORMALIZATION
    print("After normalization : ")
    knn.fit_with_norm(X_train, y_train)
    X_test, _ = normalize_dataframe(X_test, X_train)
    # Make predictions on the test set
    predictions_knn_with_norm = knn.predict(X_test)

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions_knn_with_norm)
    print("Accuracy for k = {}:".format(k_fold), accuracy)

    # Calculate precision
    precision = precision_score(y_test, predictions_knn_with_norm)
    print("Precision for k = {}:".format(k_fold), precision)

    # Calculate recall
    recall = recall_score(y_test, predictions_knn_with_norm)
    print("Recall for k = {}:".format(k_fold), recall,'\n')
    # evaluate predictions
    score_with_normalization = accuracy_score(y_test, predictions_knn_with_norm)
    k_scores_with_norm.append(score_with_normalization)
    cv_scores_with_norm.append(sum(k_scores_with_norm)/len(k_scores_with_norm))

Before Normalization : 
Accuracy for k = 1: 0.9266666666666666
Precision for k = 1: 0.9231903492391109
Recall for k = 1: 0.9289286281851103 

After normalization : 
Accuracy for k = 1: 0.8966666666666666
Precision for k = 1: 0.8997205316613212
Recall for k = 1: 0.8971835849801213 

Before Normalization : 
Accuracy for k = 3: 0.9533333333333334
Precision for k = 3: 0.9489574579831932
Recall for k = 3: 0.9535878167133005 

After normalization : 
Accuracy for k = 3: 0.93
Precision for k = 3: 0.9283037482669836
Recall for k = 3: 0.9281543411975881 

Before Normalization : 
Accuracy for k = 5: 0.9666666666666667
Precision for k = 5: 0.9647825241207595
Recall for k = 5: 0.9686683091375429 

After normalization : 
Accuracy for k = 5: 0.95
Precision for k = 5: 0.945032503476797
Recall for k = 5: 0.9501480885493075 

Before Normalization : 
Accuracy for k = 7: 0.9733333333333334
Precision for k = 7: 0.9747916666666667
Recall for k = 7: 0.974372674216908 

After normalization : 
Accuracy for k =

# Error Analysis for Classification

Some misclassified data samples were found in the results predicted by the KNN algorithm.
There are several reasons why a K-Nearest Neighbors (KNN) algorithm may misclassify some data:

    1. The KNN algorithm is sensitive to the scale of the data. If the features in the dataset have different scales, the algorithm may misclassify some data points.

    2. The choice of the value of K can also affect the performance of the algorithm. A small value of K can lead to overfitting, while a large value can lead to underfitting.

    3. The presence of noisy or irrelevant features in the dataset can also affect the performance of the algorithm.

    4. The algorithm can also be affected by the presence of outliers in the dataset.

    5. If the data is not randomly sampled and is skewed in some way, this can lead to a bias in the model and misclassification.

    6. If the training and test data is not independent and identically distributed, it can lead to poor generalization and misclassification.

The Effect of Neighbor Number (k) :

    The Effect of Neighbor Number (k) in the K-Nearest Neighbors (KNN) algorithm refers to the number of nearest neighbors used to make a prediction. This parameter is used to control the complexity of the model and the amount of smoothing required to make predictions.

    A small value of k, such as k=1, means that the algorithm will only consider the closest neighbor to make a prediction. This can lead to overfitting, as the model will be highly sensitive to noise in the training data.

    A large value of k, such as k=100, means that the algorithm will consider a larger number of neighbors to make a prediction. This can lead to underfitting, as the model will be less sensitive to noise in the training data.

    The optimal value of k will depend on the specific problem and the amount of noise in the data. A good starting point is to use a value of k that is the square root of the number of samples in the training set. However, this value should be adjusted as needed based on the results of cross-validation.

    The choice of k also affects the decision boundary of the model. When k=1, the decision boundary is irregular and complex, while larger value of k leads to a smoother decision boundary with less complex shape.

    It's worth noting that in case of a large k, the model becomes more robust to outliers.

The Effect of Normalization on the implementation of KNN algorithm :

    Normalization, also known as feature scaling, is the process of transforming the features of a dataset to a common scale. This can be important in the implementation of the K-Nearest Neighbors (KNN) algorithm, as the algorithm is sensitive to the scale of the data.

    When the features in a dataset have different scales, the algorithm may give more weight to features with larger scales and less weight to features with smaller scales. This can lead to the algorithm misclassifying some data points. By normalizing the features, the algorithm will give equal weight to all features, regardless of their scale.

    There are several ways to normalize data, the most common being min-max normalization and standardization.

    Min-max normalization scales the data to a specific range, usually between 0 and 1. The formula for min-max normalization is:

    (x - xmin) / (xmax - xmin)

    Where x is the original feature value, xmin is the minimum value of the feature in the dataset and xmax is the maximum value of the feature in the dataset.

    Standardization scales the data to have a mean of 0 and a standard deviation of 1. The formula for standardization is:

    (x - mean(x)) / std(x)

    Where x is the original feature value, mean(x) is the mean of the feature in the dataset and std(x) is the standard deviation of the feature in the dataset.

    By normalizing the data, the KNN algorithm will consider all the features on an equal footing, which can improve the accuracy of the model and reduce the chance of misclassification.

Effect of K-fold :

    K-fold cross-validation is a technique used to evaluate the performance of a machine learning model, including the K-Nearest Neighbors (KNN) algorithm. The process involves dividing the dataset into k subsets or "folds". The model is trained on k-1 of the folds and tested on the remaining fold. This process is repeated k times, with each fold being used as the test set once. The average performance of the model across all k iterations is used to estimate the model's performance on unseen data.

    The main advantage of K-fold cross-validation is that it allows for the use of all the data for both training and testing, which can provide a more accurate estimate of the model's performance. Additionally, by repeating the process k times, it provides a more robust estimate of the model's performance by averaging out any variation that might occur due to the specific samples that are used for training and testing.

    In the context of KNN algorithm, K-fold cross-validation can be used to tune the value of k, the number of nearest neighbors. By using a k-fold cross-validation, we can train the model on different subsets of the data and test it on different subsets as well. This allows us to evaluate the model's performance with different values of k and select the value that gives the best results.

Accuracy, Precision and Recall

    Accuracy, precision, and recall are commonly used metrics to evaluate the performance of a machine learning model, including the K-Nearest Neighbors (KNN) algorithm.

    Accuracy is the proportion of correctly classified instances over the total number of instances. It is calculated by dividing the number of correct predictions by the total number of predictions.

    Precision is the proportion of true positive instances among all positive instances predicted by the model. It is calculated by dividing the number of true positive predictions by the number of true positive predictions plus the number of false positive predictions.

    Recall is the proportion of true positive instances among all instances that are actually positive. It is calculated by dividing the number of true positive predictions by the number of true positive predictions plus the number of false negative predictions.

    When evaluating the performance of a KNN model, it's important to consider the balance between precision and recall. High precision means that the model is not producing many false positives, while high recall means that the model is identifying most of the positive instances. Depending on the problem and the cost of false positives vs false negatives, a trade-off between precision and recall might be desired.